In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


In [2]:
train = pd.read_csv('train_users_2.csv')

In [3]:
train.date_account_created = train.date_account_created.astype('datetime64')
train['date_first_booking'] = pd.to_datetime(train['date_first_booking'], errors='coerce')

In [4]:
#reduce the gender categories to 3 values from 4
train.gender[train.gender == 'OTHER'] = '-unknown-'

In [5]:
#reorder columns to have all features to encode side by side
cols = train.columns.tolist()
cols.insert(0,cols.pop(5))
cols.insert(0,cols.pop(15))
train = train[cols]

#one hot encoding to prepare for modelling
encoding = pd.get_dummies(train.iloc[:,6:16],columns =train.iloc[:,6:16].columns, prefix=list(train.columns[6:16]))
onehot = pd.concat([train.iloc[:,:6],encoding,train.iloc[:,16:20]],axis=1)

In [6]:
#train test split dataset to measure performance. Original test_users dataset provided by airbnb does not come with labels
# so we need to create our own test set
x_train,x_test,y_train,y_test = train_test_split(onehot,onehot['country_destination'],test_size=0.25,random_state=1)

In [7]:
#Random forest classification - instantiate classifier

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [8]:
#fit RF classifier 

clf.fit(x_train.iloc[:,6:156],y_train)

RandomForestClassifier()

In [9]:
#predict and store predictions in a series 

preds = clf.predict(x_test.iloc[:,6:156])

In [10]:
#add the series to our dataframe

x_test['predicted_country'] = preds

In [11]:
#import packages used for model evaluation

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,f1_score,recall_score

In [12]:
#get overall accuracy score for the RF model

accuracy_score(x_test['country_destination'],x_test['predicted_country'])

0.6051196521934674